In [ ]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M  # https://docs.python.org/2/library/math.html
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
from scipy.integrate import odeint

# Simple model including salinity

In [ ]:
#### Constants #####
L_x = 1000000 #zonal extent [m]
L_y = 2000000 #meridional extent [m]
A = L_x*L_y #surface area of interior [m^2] Note that the interior is probably smaller than this
P = 2*L_x+2*L_y #Perimeter [m]

g = 9.81 #gravitational acceleration [m/s^2]
C_p = 3994 #specific heat capacity of sea water [J/(kg K)] source: Steffie's paper
alpha_T = 0.2 #thermal expansion coefficient [kg m^-3 C^-1] source: Steffie's paper
alpha_S = 0.8 #Haline expansion coefficient [kg m^-3]
c = 0.006 #bottom slope coefficient
s = 0.02 #average bottom slope around the perimeter of the marginal sea
S = 35 #reference salinity [ppt]. Spall [2015]
E = -2e-8 #constant freshwater flux [m/s]
S_1 = 35.2 #from Lambert [g kg^-1]

#### Parameters ####
H = 1000 #sill height [m]
f_0 = 1.2e-4 #Coriolis parameter [s^-1]
Gamma = 20 #Restoring strength [W/(m^2 C)]

L = H/s #width of sloping topography of the boundary current [m]
T_1 = 8.87 #ingoing temperature [Celsius]
T_A = 2.8 #Atmospheric temperature [Celsius]
ro_0 = 1028  #density sea water [kg m^-3]. Range of 7 to 9 degrees of sea water
#h_1 = ... #thickness upper layer [m]

#### functions ####
epsilon = c*P/L #ratio of heat fluxed into interior by eddies compared to heat flux by advection into the basin in the inflowing boundary current [-]
mu = A*Gamma*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)) #[-]
gamma = 8*A*ro_0*f_0*S*alpha_S*E/(g*H**2*alpha_T**2*(T_1-T_A)**2)


In [ ]:
############# Computation ##################
############################################

# Main equations including salinity in the equation of state

def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
    return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
            P*H*x[6]+A*E*S,
            (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
            (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
            g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
            c*x[4]*(T_1-x[0])-x[5],
            c*x[4]*(S_1-x[2])-x[6]]

[T_0,T_2,S_0,S_2,V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

In [ ]:
[T_0,T_2,S_0,S_2,V_1,T_eddy,S_eddy]

In [ ]:
#### Follow up computations ####

#Densities -> not sure they are correct though..
ro_1 = ro_0+((alpha_T*(T_1-T_0) - alpha_S*(S_1-S_0)))
ro_out = ro_0+(alpha_T*(T_2-T_0) - alpha_S*(S_2-S_0))

#Downwelling/Medidional overturning strength at the sill [m^2/s]
W = g*H**2*(ro_1 - ro_out)/(4*ro_0*f_0)

In [ ]:
ro_1, ro_out, W

# Simple model excluding salinity

In [ ]:
#### Two main equations #####

#Interior temperature [Celsius]
T_0_simp = T_1 - (mu/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1 - T_A)

#Outflowing temperature [Celsius]
T_out_simp = T_1 - 2*mu*((T_1-T_A)/(T_1-T_0_simp)*(1+P*L/A)-1)*(T_1-T_A)

#### Follow up computations ####

#Velocity in boundary current [m/s]
V_1_simp = alpha_T*g*H*(T_1-T_0_simp)/(2*ro_0*f_0*L)

#Atmospheric heat flux/meridional heat transport at the sill [W/m^2]
Q_star = A*(T_0_simp - T_A)*Gamma + P*L*Gamma*(T_1-T_A)

#Downwelling/Medidional overturning strength at the sill [m^2/s]
W_simp = alpha_T*g*H**2*(T_1-T_0_simp)/(4*ro_0*f_0)*(1-(T_out_simp-T_0_simp)/(T_1-T_0_simp))

In [ ]:
T_0_simp, T_out_simp, V_1_simp, Q_star, W_simp

In [ ]:
T_1_array = [7,7.5,8,8.5,9,9.5,10,10.5,11]
T_0 = np.zeros(len(T_1_array))
T_2 = np.zeros(len(T_1_array))
S_0 = np.zeros(len(T_1_array))
S_2 = np.zeros(len(T_1_array))

for i in range(len(T_1_array)):
    T_1=T_1_array[i]
    def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
        return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
                P*H*x[6]+A*E*S,
                (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
                (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
                g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
                c*x[4]*(T_1-x[0])-x[5],
                c*x[4]*(S_1-x[2])-x[6]]
    [T_0[i],T_2[i],S_0[i],S_2[i],V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.plot(T_1_array, T_0, label='T_0')
plt.plot(T_1_array, T_2, label='T_2')
plt.xlabel('Inflowing temperature T_1 [C]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.plot(T_1_array, S_0, label='S_0')
plt.plot(T_1_array, S_2, label='S_1')
plt.xlabel('Inflowing temperature T_1 [C]', fontsize=14) # label along x-axes
plt.ylabel('Salinity [g/kg]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.show()

T_0_simp = np.zeros(len(T_1_array))
T_out_simp = np.zeros(len(T_1_array))
W_simp = np.zeros(len(T_1_array))

#Model comparison
for i in range(len(T_1_array)):
    T_0_simp[i] = T_1_array[i] - ((A*Gamma*f_0/(alpha_T*g*C_p*H**2*(T_1_array[i]-T_A)))/epsilon)*(M.sqrt(1+2*epsilon/mu)-1)*(T_1_array[i] - T_A)
    T_out_simp[i] = T_1_array[i] - 2*A*Gamma*f_0/(alpha_T*g*C_p*H**2*(T_1_array[i]-T_A))*((T_1_array[i]-T_A)/(T_1_array[i]-T_0_simp[i])*(1+P*L/A)-1)*(T_1_array[i]-T_A)
    W_simp[i] = alpha_T*g*H**2*(T_1_array[i]-T_0_simp[i])/(4*ro_0*f_0)*(1-(T_out_simp[i]-T_0_simp[i])/(T_1_array[i]-T_0_simp[i]))

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.plot(T_1_array, T_0, label='T_0')
plt.plot(T_1_array, T_0_simp, label='T_0_simp')
plt.xlabel('Inflowing temperature T_1 [C]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.plot(T_1_array, T_2, label='T_2')
plt.plot(T_1_array, T_out_simp, label='T_2_simp')
plt.xlabel('Inflowing temperature T_1 [C]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.show()

#Density and downwelling plots
ro_in = np.zeros(len(T_1_array))
ro_out = np.zeros(len(T_1_array))
W = np.zeros(len(T_1_array))

for i in range(len(T_1_array)):
    ro_in[i] = ro_0+((alpha_T*(T_1_array[i]-T_0[i]) - alpha_S*(S_1-S_0[i])))
    ro_out[i] = ro_0+(alpha_T*(T_2[i]-T_0[i]) - alpha_S*(S_2[i]-S_0[i]))
    W[i] = g*H**2*(ro_in[i] - ro_out[i])/(4*ro_0*f_0)

plt.figure()
plt.plot(T_1_array,W,label='W')
plt.plot(T_1_array, W_simp, label='W_simp')
plt.xlabel('Inflowing temperature T_1 [C]') # label along x-axes
plt.ylabel('Downwelling [m^2 s^(-1)]')
plt.legend(loc='lower right')
plt.show()

plt.figure()
plt.plot(T_1_array,ro_in, label='rho_in')
plt.plot(T_1_array,ro_out, label= 'rho_out')
plt.xlabel('Inflowing temperature T_1 [C]') # label along x-axes
plt.ylabel('Density [kg m^(-3)]')
plt.legend(loc='lower right')
plt.show()

In [ ]:
T_A_array = [0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7]
T_0 = np.zeros(len(T_A_array))
T_2 = np.zeros(len(T_A_array))
S_0 = np.zeros(len(T_A_array))
S_2 = np.zeros(len(T_A_array))

for i in range(len(T_A_array)):
    T_A=T_A_array[i]
    def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
        return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
                P*H*x[6]+A*E*S,
                (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
                (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
                g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
                c*x[4]*(T_1-x[0])-x[5],
                c*x[4]*(S_1-x[2])-x[6]]
    [T_0[i],T_2[i],S_0[i],S_2[i],V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.plot(T_A_array, T_0, label='T_0')
plt.plot(T_A_array, T_2, label='T_2')
plt.xlabel('Atmospheric temperature T_A [C]') # label along x-axes
plt.ylabel('Temperature [C]') # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.plot(T_A_array, S_0, label='S_0')
plt.plot(T_A_array, S_2, label='S_1')
plt.xlabel('Atmospheric temperature T_A [C]') # label along x-axes
plt.ylabel('Salinity [g/kg]') # label along x-axes
plt.legend(loc='lower right')
plt.suptitle('Varying atmospheric temperature T_A')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=None)
plt.show()

ro_in = np.zeros(len(T_A_array))
ro_out = np.zeros(len(T_A_array))
W = np.zeros(len(T_A_array))


for i in range(len(T_A_array)):
    ro_in[i] = ro_0+((alpha_T*(T_1-T_0[i]) - alpha_S*(S_1-S_0[i])))
    ro_out[i] = ro_0+(alpha_T*(T_2[i]-T_0[i]) - alpha_S*(S_2[i]-S_0[i]))
    W[i] = g*H**2*(ro_in[i] - ro_out[i])/(4*ro_0*f_0)

plt.figure()
plt.plot(T_A_array,W)
plt.xlabel('Atmospheric temperature T_A [C]') # label along x-axes
plt.ylabel('Downwelling [m^2 s^(-1)]')
plt.show()

plt.figure()
plt.plot(T_A_array,ro_in, label='rho_in')
plt.plot(T_A_array,ro_out, label= 'rho_out')
plt.xlabel('Atmospheric temperature T_A [C]') # label along x-axes
plt.ylabel('Density [kg m^(-3)]')
plt.legend(loc='lower right')
plt.show()

In [ ]:
S_1_array = [34.5,34.6,34.7,34.8,34.9,35,35.1,35.2,35.3,35.4,35.5,35.6]
T_0 = np.zeros(len(S_1_array))
T_2 = np.zeros(len(S_1_array))
S_0 = np.zeros(len(S_1_array))
S_2 = np.zeros(len(S_1_array))

for i in range(len(S_1_array)):
    S_1=S_1_array[i]
    def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
        return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
                P*H*x[6]+A*E*S,
                (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
                (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
                g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
                c*x[4]*(T_1-x[0])-x[5],
                c*x[4]*(S_1-x[2])-x[6]]
    [T_0[i],T_2[i],S_0[i],S_2[i],V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.scatter(S_1_array, T_0, label='T_0')
plt.scatter(S_1_array, T_2, label='T_2')
plt.xlabel('Inflowing salinity [g kg^(-1)]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.scatter(S_1_array, S_0, label='S_0')
plt.scatter(S_1_array, S_2, label='S_1')
plt.xlabel('Inflowing salinity [g kg^(-1)]', fontsize=14) # label along x-axes
plt.ylabel('Salinity [g/kg]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.show()

ro_in = np.zeros(len(S_1_array))
ro_out = np.zeros(len(S_1_array))
W = np.zeros(len(S_1_array))


for i in range(len(S_1_array)):
    ro_in[i] = ro_0+((alpha_T*(T_1-T_0[i]) - alpha_S*(S_1_array[i]-S_0[i])))
    ro_out[i] = ro_0+(alpha_T*(T_2[i]-T_0[i]) - alpha_S*(S_2[i]-S_0[i]))
    W[i] = g*H**2*(ro_in[i] - ro_out[i])/(4*ro_0*f_0)

plt.figure()
plt.plot(S_1_array,W)
plt.xlabel('Inflowing salinity [g kg^(-1)]', fontsize=14) # label along x-axes
plt.ylabel('Downwelling [m^2 s^(-1)]')
plt.show()

plt.figure()
plt.plot(S_1_array,ro_in, label='rho_in')
plt.plot(S_1_array,ro_out, label= 'rho_out')
plt.xlabel('Inflowing salinity [g kg^(-1)]', fontsize=14) # label along x-axes
plt.ylabel('Density [kg m^(-3)]')
plt.legend(loc='lower right')
plt.show()

In [ ]:
Gamma_array = [10,20,30,40,50,60]
T_0 = np.zeros(len(Gamma_array))
T_2 = np.zeros(len(Gamma_array))
S_0 = np.zeros(len(Gamma_array))
S_2 = np.zeros(len(Gamma_array))

for i in range(len(Gamma_array)):
    Gamma=Gamma_array[i]
    def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
        return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
                P*H*x[6]+A*E*S,
                (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
                (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
                g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
                c*x[4]*(T_1-x[0])-x[5],
                c*x[4]*(S_1-x[2])-x[6]]
    [T_0[i],T_2[i],S_0[i],S_2[i],V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.plot(Gamma_array, T_0, label='T_0')
plt.plot(Gamma_array, T_2, label='T_2')
plt.xlabel('Restoring strength [W/(m^2 K)]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.plot(Gamma_array, S_0, label='S_0')
plt.plot(Gamma_array, S_2, label='S_2')
plt.xlabel('Restoring strength [W/(m^2 K)]', fontsize=14) # label along x-axes
plt.ylabel('Salinity [g/kg]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.show()

T_0_simp = np.zeros(len(Gamma_array))
T_out_simp = np.zeros(len(Gamma_array))
W_simp = np.zeros(len(Gamma_array))

#Model comparison
for i in range(len(Gamma_array)):
    T_0_simp[i] = T_1 - ((A*Gamma_array[i]*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)))/epsilon)*(M.sqrt(1+2*epsilon/A*Gamma_array[i]*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)))-1)*(T_1 - T_A)
    T_out_simp[i] = T_1 - 2*(A*Gamma_array[i]*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)))*((T_1-T_A)/(T_1-T_0_simp[i])*(1+P*L/A)-1)*(T_1-T_A)
    W_simp[i] = alpha_T*g*H**2*(T_1-T_0_simp[i])/(4*ro_0*f_0)*(1-(T_out_simp[i]-T_0_simp[i])/(T_1-T_0_simp[i]))

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.plot(Gamma_array, T_0, label='T_0')
plt.plot(Gamma_array, T_0_simp, label='T_0_simp')
plt.xlabel('Restoring strength [W/(m^2 K)]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.plot(Gamma_array, T_2, label='T_2')
plt.plot(Gamma_array, T_out_simp, label='T_2_simp')
plt.xlabel('Restoring strength [W/(m^2 K)]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.show()

#Density plots
ro_in = np.zeros(len(Gamma_array))
ro_out = np.zeros(len(Gamma_array))
W = np.zeros(len(Gamma_array))


for i in range(len(Gamma_array)):
    ro_in[i] = ro_0+((alpha_T*(T_1-T_0[i]) - alpha_S*(S_1-S_0[i])))
    ro_out[i] = ro_0+(alpha_T*(T_2[i]-T_0[i]) - alpha_S*(S_2[i]-S_0[i]))
    W[i] = g*H**2*(ro_in[i] - ro_out[i])/(4*ro_0*f_0)

plt.figure()
plt.plot(Gamma_array, W_simp, label='W_simp')
plt.plot(Gamma_array,W, label='W')
plt.xlabel('Restoring strength [W/(m^2 K)]', fontsize=14) # label along x-axes
plt.ylabel('Downwelling [m^2 s^(-1)]')
plt.legend(loc='lower right')
plt.show()

plt.figure()
plt.plot(Gamma_array,ro_in, label='rho_in')
plt.plot(Gamma_array,ro_out, label= 'rho_out')
plt.xlabel('Restoring strength [W/(m^2 K)]', fontsize=14) # label along x-axes
plt.ylabel('Density [kg m^(-3)]')
plt.legend(loc='lower right')
plt.show()

In [ ]:
H_array = [500,600,700,800,900,1000,1100,1200,1300]
T_0 = np.zeros(len(H_array))
T_2 = np.zeros(len(H_array))
S_0 = np.zeros(len(H_array))
S_2 = np.zeros(len(H_array))

for i in range(len(H_array)):
    H=H_array[i]
    def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
        return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
                P*H*x[6]+A*E*S,
                (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
                (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
                g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
                c*x[4]*(T_1-x[0])-x[5],
                c*x[4]*(S_1-x[2])-x[6]]
    [T_0[i],T_2[i],S_0[i],S_2[i],V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.scatter(H_array, T_0, label='T_0')
plt.scatter(H_array, T_2, label='T_2')
plt.xlabel('Sill height [m]', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.scatter(H_array, S_0, label='S_0')
plt.scatter(H_array, S_2, label='S_1')
plt.xlabel('Sill height [m]', fontsize=14) # label along x-axes
plt.ylabel('Salinity [g/kg]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.show()

ro_in = np.zeros(len(H_array))
ro_out = np.zeros(len(H_array))
W = np.zeros(len(H_array))


for i in range(len(H_array)):
    ro_in[i] = ro_0+((alpha_T*(T_1-T_0[i]) - alpha_S*(S_1-S_0[i])))
    ro_out[i] = ro_0+(alpha_T*(T_2[i]-T_0[i]) - alpha_S*(S_2[i]-S_0[i]))
    W[i] = g*H_array[i]**2*(ro_in[i] - ro_out[i])/(4*ro_0*f_0)

plt.figure()
plt.plot(H_array,W)
plt.xlabel('Sill height [m]', fontsize=14) # label along x-axes
plt.ylabel('Downwelling [m^2 s^(-1)]')
plt.show()

plt.figure()
plt.plot(H_array,ro_in, label='rho_in')
plt.plot(H_array,ro_out, label= 'rho_out')
plt.xlabel('Sill height [m]', fontsize=14) # label along x-axes
plt.ylabel('Density [kg m^(-3)]')
plt.legend(loc='lower right')
plt.show()

In [ ]:
c_array = [0.001,0.002,0.003,0.004,0.005,0.006, 0.007,0.008,0.009,0.01]
T_0 = np.zeros(len(c_array))
T_2 = np.zeros(len(c_array))
S_0 = np.zeros(len(c_array))
S_2 = np.zeros(len(c_array))

for i in range(len(c_array)):
    c=c_array[i]
    def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
        return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro_0*C_p),
                P*H*x[6]+A*E*S,
                (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro_0*C_p),
                (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
                g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
                c*x[4]*(T_1-x[0])-x[5],
                c*x[4]*(S_1-x[2])-x[6]]
    [T_0[i],T_2[i],S_0[i],S_2[i],V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

plt.figure(figsize=(9, 3))

plt.subplot(121)
plt.plot(c_array, T_0, label='T_0')
plt.plot(c_array, T_2, label='T_2')
plt.xlabel('eddy coefficient', fontsize=14) # label along x-axes
plt.ylabel('Temperature [C]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.subplot(122)
plt.plot(c_array, S_0, label='S_0')
plt.plot(c_array, S_2, label='S_2')
plt.xlabel('eddy coefficient', fontsize=14) # label along x-axes
plt.ylabel('Salinity [g/kg]', fontsize=14) # label along x-axes
plt.legend(loc='lower right')
plt.show()

In [ ]:
ro_in = np.arange(1027,1030,0.1)
ro_out = np.arange(1027,1030,0.1)
X,Y = np.meshgrid(ro_in,ro_out)
W = g*H**2*(X - Y)/(4*ro_0*f_0)

fig, ax = plt.subplots(figsize=(9,9))
plt.contourf(X,Y,W)
plt.colorbar(label='[m^2/s]')
plt.xlabel("rho_in")
plt.ylabel("rho_out")
plt.title("Contour plot of the downwelling as a function of in- and outflowing density")
plt.show()


# Spatial Model

In [ ]:

############ Initilisation #################
############################################

#### Constants #####
L_x = 1e6          # [m] zonal extent 
L_y = 2e6          # [m] meridional extent 
A = L_x*L_y        # [m^2] surface area of interior 
P = 2*L_x+2*L_y    # [m] Perimeter 

g = 9.81           # [m/s^2] gravitational acceleration 
c = 0.006          # [] bottom slope coefficient
s = 0.02           # [] average bottom slope around the perimeter of the marginal sea

c_theta = 0.007    # [] eddy coefficient halocline
alpha = 1e-4       # [1/C] thermal expansion coefficient
C_p = 4.2e6        # [J/(m^3*C)] specific heat capacity of sea water 
H_t = 1e3          # [m] depth interior
H_s = 2e2          # [m] depth shelves
W_c = 1e5          # [m] width slope
rho_0 = 1028       # [kg m^-3] reference density sea water 

f_0 = 1.4e-4       # [s^-1] Coriolis parameter 
Gamma = 20         # [W/(m^2 C)] Restoring strength 
T_A = 2           # [Celsius] Atmospheric temperature 

#### Initilisation ####
steps=10# number of spatial steps
Dx=P/steps         # size of spatial steps

T_0 = np.zeros(steps)
H_1 = np.zeros(steps)
T_1 = np.zeros(steps)

T_1[0] = 8         # [C] Inflow AW temperature
H_1[0] = 600       # [m] Inflow AW thickness
T_0[0] = 4         # [C] First guess interior DW temperature

In [ ]:
integral=0.0
for i in range(1,steps-1):
    def main(x): # function with six unkowns:  T_1,  H_1, kappa_theta, psi_1, Q_1, delta_rho_theta
                 #                             x[0]  x[1]     x[2]      x[3]  x[4]      x[5]
        return [x[2]-(c*g*x[5]*(x[1]**2)*(H_t-H_s))/(2*f_0*rho_0*W_c*(H_t-x[1])),
                C_p*x[3]*((x[0]-T_1[i-1])/Dx)+x[4]*W_c,
                ((x[1]-H_1[i-1])/Dx)+((f_0*x[2])/(g*alpha*(x[0]-T_0[i-1])*x[1]))-((Gamma*W_c*f_0*(x[0]-T_A))/(C_p*g*alpha*(x[0]-T_0[i-1])**2)*x[1]),
                x[3]-(g*x[5]*x[1]**2)/(2*f_0*rho_0),
                x[4]-Gamma*(x[0]-T_A),
                x[5]-rho_0*alpha*(x[0]-T_0[i-1])]
    [T_1[i], H_1[i], kappa_theta, psi_1, Q_1, delta_rho_theta] = fsolve(main, [1,1,1,1,1,1]) 
    integral=integral+(kappa_theta*(T_1[i]-T_0[i-1]))*Dx
    T_0[i]=T_A+(C_p)/(Gamma*A)*integral

In [ ]:
x=np.linspace(0,steps*Dx,steps)

plt.subplot(131)
plt.plot(x, T_1, label='$T_1$')
plt.xlabel('length x [m]')
plt.ylabel('temperature [C]')
plt.legend(loc='upper right')
plt.subplot(132)
plt.plot(x, H_1, label='$H_1$')
plt.xlabel('length x [m]')
plt.ylabel('depth [m]')
plt.legend(loc='upper right')
plt.subplot(133)
plt.plot(x, T_0, label='$T_0$')
plt.xlabel('length x [m]')
plt.ylabel('depth [m]')
plt.legend(loc='upper right')
plt.suptitle('simple plot')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.3, hspace=None)
plt.show()